# Setup

 Import standard modules

In [4]:
%load_ext autoreload
%autoreload 2
%matplotlib widget
import os
import numpy as np
import pandas as pd
import time
import seaborn as sns
sns.set()
from matplotlib import pyplot as plt
from matplotlib.widgets import MultiCursor
from scipy.signal import argrelmin, argrelmax
from scipy.interpolate import InterpolatedUnivariateSpline
from sklearn.linear_model import LinearRegression
import sys
sys.path.append("..")
from src import definitions as defs
from src import features, models

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
wdir = defs.ROOT / "data" / "interim"

Import project modules from src folder

In [6]:
MSL = 1.12
beta_mean = 6.25e-3
beta_mean_high = 14.5e-3
beta_mean_low = -2e-3
tides = features.load_tide(wdir, "tides-50yr-30s.feather")
tides = tides.to_frame(name="elevation")
tides["elapsed_sec"] = (tides.index - tides.index[0]).total_seconds().astype(int)
tides = tides[["elapsed_sec", "elevation"]]

lm1 = LinearRegression()
X1 = ((tides.index - tides.index[0]).total_seconds().values / pd.Timedelta(days=365).total_seconds()).reshape(-1, 1)
Y1 = tides.elevation.values.reshape(-1, 1)
lm1.fit(X1, Y1)
beta_mean_high = beta_mean_high - lm1.coef_[0][0]
beta_mean_low = beta_mean_low - lm1.coef_[0][0]

highs = tides.loc[tides.index[argrelmax(tides.elevation.values)[0]]]
lows = tides.loc[tides.index[argrelmin(tides.elevation.values)[0]]]

tides_d2dt = np.gradient(tides.elevation.values, 30.0)
inflect_iloc = np.concatenate((argrelmax(tides_d2dt)[0], argrelmin(tides_d2dt)[0]))
inflect_iloc.sort(kind="mergesort")
inflects = tides.loc[tides.index[inflect_iloc]]

In [7]:
start = tides.index[0]
end = start + pd.DateOffset(years=1)
subset = tides[start:end]

subset_highs = subset.iloc[argrelmax(subset.elevation.values)[0]]
subset_lows = subset.iloc[argrelmin(subset.elevation.values)[0]]
subset_extrema = pd.concat([subset_highs, subset_lows]).sort_index()

tides_d2dt = np.gradient(subset.elevation.values, 30.0)

new_highs = highs.elevation + beta_mean_high * highs.elapsed_sec / pd.Timedelta(days=365).total_seconds()
new_lows = lows.elevation + beta_mean_low * lows.elapsed_sec / pd.Timedelta(days=365).total_seconds()

In [8]:
highs["elevation_new"] = new_highs
lows["elevation_new"] = new_lows
nodes = pd.concat([highs, lows, inflects], keys=["highs", "lows", "inflects"], names=["type"]).reset_index(level=[0]).sort_index()

In [9]:
lm2 = LinearRegression()
X2 = ((new_highs.index - tides.index[0]).total_seconds().values / pd.Timedelta(days=365).total_seconds()).reshape(-1, 1)
Y2 = new_highs.values.reshape(-1, 1)
lm2.fit(X2, Y2)
new_highs_pred = lm2.predict(X2).reshape(-1)
new_highs_label = "$\zeta_{{HT}}={:.4f}t {:+.4f}$".format(lm2.coef_[0][0], lm2.intercept_[0])

lm3 = LinearRegression()
X3 = ((new_lows.index - tides.index[0]).total_seconds().values / pd.Timedelta(days=365).total_seconds()).reshape(-1, 1)
Y3 = new_lows.values.reshape(-1, 1)
lm3.fit(X3, Y3)
new_lows_pred = lm3.predict(X3).reshape(-1)
new_lows_label = "$\zeta_{{LT}}={:.4f}t {:+.4f}$".format(lm3.coef_[0][0], lm3.intercept_[0])

fig = plt.figure(figsize=(10, 5))
ax = sns.scatterplot(x=highs.index, y=new_highs, marker=".", color="forestgreen", edgecolor=None)
sns.lineplot(ax=ax, x=highs.index, y=new_highs_pred, color="forestgreen", label=new_highs_label)
sns.scatterplot(ax=ax, x=lows.index, y=new_lows, marker=".", color="tomato", edgecolor=None)
sns.lineplot(ax=ax, x=lows.index, y=new_lows_pred, color="tomato", label=new_lows_label)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<AxesSubplot:xlabel='datetime'>

In [10]:
start = subset.index[0]
stop = start + pd.Timedelta(days=7)
sample = tides.loc[start:stop].reset_index()
sample_h = highs.loc[start:stop].reset_index()
sample_l = lows.loc[start:stop].reset_index()
sample_inf = inflects.loc[start:stop].reset_index()

window = highs.reset_index().datetime.diff().mean()
roll_mean = sample.rolling(window=window, on="datetime").mean()

fig = plt.figure(figsize=(10, 5))
ax = plt.axes()

sns.lineplot(data=sample, x="datetime", y="elevation", color="cornflowerblue", alpha=0.5, label="Tides", ax=ax,)
sample_h.plot.scatter(x="datetime", y="elevation", color="green", label="Highs", ax=ax)
sample_l.plot.scatter(x="datetime", y="elevation", color="red", label="Lows", ax=ax)
sns.scatterplot(data=sample_inf, x="datetime", y="elevation", ax=ax, color="black")
ax.margins(x=0.0)
ax.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [11]:
starts = nodes.reset_index().datetime.to_frame(name="start")
stops = nodes.reset_index().datetime.shift(-1).to_frame(name="stop")
df = starts[:-1].join(stops[:-1])
df["method"] = np.tile([1,2,3,4], reps=int(len(nodes) / 4))

In [12]:
df.loc[(df.method==1) | (df.method==3), "inflect"] = nodes.loc[df[(df.method==1) | (df.method==3)].start].elevation.values
df.loc[(df.method==1) | (df.method==3), "extrema"] = nodes.loc[df[(df.method==1) | (df.method==3)].stop].elevation.values
df.loc[(df.method==1) | (df.method==3), "extrema_new"] = nodes.loc[df[(df.method==1) | (df.method==3)].stop].elevation_new.values

df.loc[(df.method==2) | (df.method==4), "inflect"] = nodes.loc[df[(df.method==2) | (df.method==4)].stop].elevation.values
df.loc[(df.method==2) | (df.method==4), "extrema"] = nodes.loc[df[(df.method==2) | (df.method==4)].start].elevation.values
df.loc[(df.method==2) | (df.method==4), "extrema_new"] = nodes.loc[df[(df.method==2) | (df.method==4)].start].elevation_new.values

df["difference"] = df.extrema_new - df.extrema

In [ ]:
amplified_tides = []
for row in df.itertuples():
    if row.method == 1 or row.method == 2:
        old = tides.loc[row.start:row.stop].elevation
        weights = (old - old.min()) / (old.max() - old.min())
        new = old + weights * row.difference
    elif row.method == 3 or row.method == 4:
        old = tides.loc[row.start:row.stop].elevation
        weights = (old.max() - old) / (old.max() - old.min())
        new = old + weights * row.difference
    amplified_tides.append(new)
    
amplified_tides = pd.concat(amplified_tides).drop_duplicates().to_frame(name="elevation")

lm4 = LinearRegression()
X4 = ((amplified_tides.index - amplified_tides.index[0]).total_seconds().values / pd.Timedelta(days=365).total_seconds()).reshape(-1, 1)
Y4 = amplified_tides.elevation.values.reshape(-1, 1)
lm4.fit(X4, Y4)
amplified_tides["lm"] = lm4.predict(X4).reshape(-1)
amplified_tides_label = "$\zeta_{{MW}}={:.4f}t {:+.4f}$".format(lm4.coef_[0][0], lm4.intercept_[0])

In [ ]:
plt.close("all")
i = amplified_tides.index[0]
# ii = i + pd.Timedelta(days=365)
ii = amplified_tides.index[-1]

subset = amplified_tides.loc[i:ii]
subset_highs = subset.loc[subset.index[argrelmax(subset.elevation.values)[0]]]
subset_lows = subset.loc[subset.index[argrelmin(subset.elevation.values)[0]]]

subset_lm = LinearRegression()
subset_x = ((subset.index - subset.index[0]).total_seconds().values / pd.Timedelta(days=365).total_seconds()).reshape(-1, 1)
subset_lm.fit(subset_x, subset.elevation.values.reshape(-1, 1))

subset_highs_lm = LinearRegression()
subset_highs_x = ((subset_highs.index - subset.index[0]).total_seconds().values / pd.Timedelta(days=365).total_seconds()).reshape(-1, 1)
subset_highs_lm.fit(subset_highs_x, subset_highs.elevation.values.reshape(-1, 1))

subset_lows_lm = LinearRegression()
subset_lows_x = ((subset_lows.index - subset.index[0]).total_seconds().values / pd.Timedelta(days=365).total_seconds()).reshape(-1, 1)
subset_lows_lm.fit(subset_lows_x, subset_lows.elevation.values.reshape(-1, 1))

n = pd.Timedelta(minutes=60)
subset = subset.resample(n).first()
num_index = ((subset.index - subset.index[0]).total_seconds().values / pd.Timedelta(days=365).total_seconds()).reshape(-1, 1)

fig = plt.figure(figsize=(10, 5))
ax = plt.axes()

sns.scatterplot(ax=ax, x=subset.index, y=subset.elevation, color="cornflowerblue", edgecolor=None, alpha=0.3)
sns.scatterplot(ax=ax, x=subset_highs.index, y=subset_highs.elevation, color="forestgreen", edgecolor=None)
sns.scatterplot(ax=ax, x=subset_lows.index, y=subset_lows.elevation, color="tomato", edgecolor=None)

sns.lineplot(ax=ax, x=subset.index, y=subset_lm.predict(num_index).reshape(-1), color="white", linewidth=3.5)
sns.lineplot(ax=ax, x=subset.index, y=subset_lm.predict(num_index).reshape(-1), color="cornflowerblue", linewidth=3)


sns.lineplot(ax=ax, x=subset.index, y=subset_highs_lm.predict(num_index).reshape(-1), color="white", linewidth=3.5)
sns.lineplot(ax=ax, x=subset.index, y=subset_highs_lm.predict(num_index).reshape(-1), color="forestgreen", linewidth=3)

sns.lineplot(ax=ax, x=subset.index, y=subset_lows_lm.predict(num_index).reshape(-1), color="white", linewidth=3.5)
sns.lineplot(ax=ax, x=subset.index, y=subset_lows_lm.predict(num_index).reshape(-1), color="tomato", linewidth=3)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [18]:
from src import models
ts = features.load_tide(wdir, "tides-10yr-30s.feather") + MSL
tf = models.TidalFlat(
    tides=ts,
    land_elev_init=2.6,
    conc_bound=0.3,
    grain_diam=3.0e-5,
    grain_dens=2.65e3,
    bulk_dens=900,
    org_rate_yr=2e-4,
    comp_rate_yr=4e-3,
    sub_rate_yr=3e-3,
)
tf.run()


Progress:   0%|          | 0/3652 [00:00<?, ?day/s]

----------------------------------------
Starting elevation:       2.600       m
Final elevation:          2.780       m
Elevation change:         18.042     cm
----------------------------------------
Aggradation:              24.695     cm
Degradation:              6.801      cm
----------------------------------------
Runtime:                    03 min 36 s


In [22]:
plt.close("all")
tf.plot(frac=0.01)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [23]:
plt.close("all")
tf.inundations[1].plot()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [44]:
plt.close("all")
df = tf.results.set_index("datetime")
df["land_elev_delta"] = df.land_elev.diff()
df["land_elev_delta_yearly_avg"] = df.land_elev_delta.rolling(window=pd.offsets.Day(365)).sum()
df["land_elev_delta_yearly_avg"].plot()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<AxesSubplot:xlabel='datetime'>